## Final version
For the final version, we need to do a few things after the integration:
1) Use base h5ad file to regain all genes so we don't lose that information. 
2) Add the integrated latent space and cell type annotation. 
3) Harmonize the labels. 
4) Use these for label prediction. 


In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/hpc/hers_basak/bin/miniconda3/envs/sc2_seurat/bin/python
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:40:17) 
[GCC 9.4.0]
sys.version_info(major=3, minor=9, micro=9, releaselevel='final', serial=0)


In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))#embed HTML within IPython notebook and make containers 90% of screen
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import seaborn as sns
import os
import matplotlib.pyplot as plt
#import scvi
import torch

%matplotlib inline
sc.settings.verbosity = 0             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, facecolor='white') #figure resolution and background color

#To make reproducible plots
np.random.seed(41)
# This is used
# os.path.join('data/'+object_names[i][6:10]+'_preprocessed.h5ad')
#sc.logging.print_versions()

Functions we will need

In [3]:
#Used in Jupyter Notebook so is loosely coupled (does not import dependencies on its own)

## Very hard-coded function that prepares the various resolution labels based on the original multi-level labels
# adata: your Anndataobject with cell_type_lvl1, cell_type_lvl2 and cell_type_lvl3
# Return: none, it will add .obs['cell_type_lowest_res'], .obs['cell_type_low_res'], .obs['cell_type_med_res'] and .obs['cell_type_high_res'] to existing adata

def prepare_labels(adata):    
    low=adata.obs['cell_type_lvl1'].tolist()
    med=adata.obs['cell_type_lvl2'].tolist()
    high=adata.obs['cell_type_lvl3'].tolist()
    atlas=adata.obs['atlas'].tolist()

    #Siletti supercluster resolution was too high
    siletti_supercluster=['Miscellaneous', 'Microglia', 'Vascular', 'Fibroblast', 'Oligodendrocyte precursor', 'Committed oligodendrocyte precursor', 'Oligodendrocyte', 'Bergmann glia', 'Astrocyte', 'Ependymal', 'Choroid plexus', 'Deep-layer near-projecting', 'Deep-layer corticothalamic and 6b', 'Hippocampal CA1-3', 'Upper-layer intratelencephalic', 'Deep-layer intratelencephalic', 'Amygdala excitatory', 'Hippocampal CA4', 'Hippocampal dentate gyrus', 'Medium spiny neuron', 'Eccentric medium spiny neuron', 'Splatter', 'MGE interneuron', 'LAMP5-LHX6 and Chandelier', 'CGE interneuron', 'Upper rhombic lip', 'Cerebellar inhibitory', 'Lower rhombic lip', 'Mammillary body', 'Thalamic excitatory', 'Midbrain-derived inhibitory']

    siletti_class=['MISC', 'MGL', 'ENDO', 'FIB', 'OPC', 'OLIGO', 'OLIGO', 'ASTRO', 'ASTRO', 'EPEN', 'CHRP', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR','NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR', 'NEUR']

    cell_type_lowest_res = list()
    cell_type_low_res = list()
    cell_type_med_res = list()
    cell_type_high_res = list()

    for i in range(len(adata.obs_names)):

        #check for nan values
        if low[i] in ['','None']:
            cell_type_lowest_res.append('Unknown') #lowest
            cell_type_low_res.append('Unknown') #low
            cell_type_med_res.append('Unknown') #med
            cell_type_high_res.append('Unknown') #high
        else:
            #low for all because it is the same as lvl1
            cell_type_low_res.append(str(low[i]).replace('_','-')) #low

            #high and med for all except siletti
            if atlas[i]!='siletti':
                if high[i]!= '':
                    cell_type_high_res.append(str(high[i]).replace('_','-'))#high
                    cell_type_med_res.append(str(high[i]).replace('_','-'))#med
                elif med[i]!= '':
                    cell_type_high_res.append(str(med[i]).replace('_','-'))#high
                    cell_type_med_res.append(str(med[i]).replace('_','-'))#med
                else:
                    cell_type_high_res.append(str(low[i]).replace('_','-'))#high
                    cell_type_med_res.append(str(low[i]).replace('_','-'))#med

            #lowest, med and high for siletti
            if atlas[i]=='siletti':
                cell_index=siletti_supercluster.index(low[i])
                cell_type_lowest_res.append(str(siletti_class[cell_index]).replace('_','-'))#lowest
                cell_type_high_res.append(str(high[i]).replace('_','-'))#high
                cell_type_med_res.append(str(med[i]).replace('_','-'))#med

            #lowest for all others, because labels are too high resolution
            elif atlas[i]=='welch':

                if 'ASTRO' in low[i]:
                    cell_type_lowest_res.append('ASTRO')#lowest
                elif 'ENDO' in low[i]:
                    cell_type_lowest_res.append('ENDO')#lowest
                elif 'NEURO' in low[i]:
                    cell_type_lowest_res.append('NEURO')#lowest
                elif 'OLIGO' in low[i]:
                    cell_type_lowest_res.append('OLIGO')#lowest
                elif 'OPC' in low[i]:
                    cell_type_lowest_res.append('OPC')#lowest
                elif 'MG' in low[i]:
                    cell_type_lowest_res.append('MG')#lowest
            elif atlas[i]=='smajic':
                if 'DaNs' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                elif 'CADPS2+ neurons' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                elif 'Excitatory' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                elif 'GABA' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                elif 'Inhibitory' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                else:
                    cell_type_lowest_res.append(low[i])#lowest
            elif atlas[i]=='agarwal':
                if 'DaN' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                elif 'GABA neurons' in low[i]:
                    cell_type_lowest_res.append('Neuron')#lowest
                else:
                    cell_type_lowest_res.append(low[i])#lowest
            else:
                cell_type_lowest_res.append(str(low[i]).replace('_','-'))#lowest
                


    adata.obs['cell_type_lowest_res'] = cell_type_lowest_res
    adata.obs['cell_type_low_res'] = cell_type_low_res
    adata.obs['cell_type_med_res'] = cell_type_med_res
    adata.obs['cell_type_high_res'] = cell_type_high_res

    adata.obs['cell_type_high_res'] = np.char.add(np.char.add(np.array(adata.obs['cell_type_high_res'], dtype= str), '-'),
                                                 np.array(adata.obs['atlas'], dtype=str))

    adata.obs['cell_type_med_res'] = np.char.add(np.char.add(np.array(adata.obs['cell_type_med_res'], dtype= str), '-'),
                                                 np.array(adata.obs['atlas'], dtype=str))

    adata.obs['cell_type_low_res'] = np.char.add(np.char.add(np.array(adata.obs['cell_type_low_res'], dtype= str), '-'),
                                                 np.array(adata.obs['atlas'], dtype=str))

    adata.obs['cell_type_lowest_res'] = np.char.add(np.char.add(np.array(adata.obs['cell_type_lowest_res'], dtype= str), '-'),
                                                 np.array(adata.obs['atlas'], dtype=str))


In [4]:
##Harmonizes Labels within a dataset according to a hierarchy tree 
#tried to make it work for all level, but only works for level 1...
#Used in Jupyter Notebook [tightly-coupled/independent]

def descend(nodes): 
    """Get the right resolution of labels -> List(str)
    
    Parameters
    -------------------------------------------------------------
    nodes: List[node]
        List of descendants nodes of the label node that you want to use as harmonized label
    
    return
    -----------------------------------------------------------------
    List of nested lists that represent all the node names that fall under the first layer of nodes.
    """
    labels=list()
    for node in nodes:
        for name in node.name:
            labels.append(name)
        if node.descendants==[]:
            continue
        else:         
            for name in descend(node.descendants):
                labels.append(name)
    return labels
    
    
def get_label_descendants(tree,level):
    """Gets the descendants at a certain level for all branches and subbranches
    
    Parameters
    --------------------------------------------------------------------------
    tree: TreeNode 
            Classification tree to take equivalent labels from (resulting from learn_tree method)
    level: Int
            Which tree depth to use to harmonize the labels (0=root only, 1: first descendants)
    
    return
    -----------------------------------------------------
    A list of nodes at a certain level
    """
    nodes=list()
    if level == 0:
        return tree
    for branch in tree.descendants:
        nodes.append(get_label_descendants(branch,level-1))
    return nodes


    
def harmonize_labels(adata,label_key,tree,level,new_key='harmonized_cell_type'):
    
    """Changes the labels to equivalent onces in all batches.
    
    Parameters
    ------------------------------------------------------
    adata: Anndata object
            Object containing the expression data as well as annotations and latent space
    label_key: str
            String for the key of the cell_type annotation that needs to be harmonized
    tree: TreeNode 
            Classification tree to take equivalent labels from (resulting from learn_tree method)
    batches: List[str]
            List of batch keys (also used for priority order in which labels is taken, need to have batch keys as suffix)
    level: Int
            Which tree depth to use to harmonize the labels (0=root only, 1: first descendants)
    new_key: str
            The new key where the harmonized cell labels will be
    return
    -------------------------------------------------------
    None, but original Anndata object will gain .obs['harmonized_celltype'] feature
    """
    
    descendants = get_label_descendants(tree[0],level)
    old_labels=list()
    new_labels=list()
    for node in descendants:
        old_labels.append(descend(node.descendants))
        node_name=node.name
        for name in node_name:
            old_labels[-1].append(name)
        new_labels.append(node_name[0])
    
    new_values=list()
    for original in adata.obs[label_key].tolist():
        unknown=True
        for i in range(len(old_labels)):
            if original in old_labels[i]:
                new_values.append(new_labels[i])
                unknown=False
                break
        if unknown ==True:
            new_values.append('unknown')
            
                
    adata.obs[new_key]=new_values
    
    return adata
    

In [5]:
#set data path
os.chdir('/home/hers_basak/jjiang/jack/')

(1) Need 2 files:
- Base: .h5ad  file with all the genes/ the amount of genes you want.
- Annotation: .h5ad file with annotation and integration latent space.

We will be adding the annotations from the integrated to the base.

In [6]:
base= sc.read_h5ad('/home/hers_basak/jjiang/jack/for_next_person/data/complete_base.h5ad')

In [7]:
annotation=sc.read_h5ad('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/adata.h5ad')

In [8]:
base

AnnData object with n_obs × n_vars = 522220 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res'
    uns: 'atlas_colors'
    obsm: 'X_

In [9]:
annotation

AnnData object with n_obs × n_vars = 522220 × 2000
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', '_scvi_batch', '_scvi_labels', 'leiden', 'cell_type_low_res', 'cell_type_high_res', 'harmonized_cell_type', 'harmonized_cell_

!!!Forgot to add the missing welch annotation (due to inconsistency in their identifiers, my code didnt capture three samples' annotation). Welch only has one level of annotation stored in "original_celltype".

In [10]:
for i in ['original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3']:
    base.obs[i] = annotation.obs[i]

(2) Adding the celltype annotation to the base

In [11]:
prepare_labels(base)

In [40]:
base

AnnData object with n_obs × n_vars = 522220 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res', 'harmonized_lowest_res', 'harmonized

In [54]:
base.write('/home/hers_basak/jjiang/jack/for_next_person/data/complete_base.h5ad')

(2) Adding latent space

In [13]:
base.obsm['X_scVI']=annotation.obsm['X_scVI']

(3) Harmonize the labels using the tree.

In [14]:
import pickle
with open('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/tree_ref_lowest.pkl', 'rb') as file: 
      
    tree_ref = pickle.load(file)

/hpc/hers_basak/bin/miniconda3/envs/sc2_seurat/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.3.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/hpc/hers_basak/bin/miniconda3/envs/sc2_seurat/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator PCA from version 1.3.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
tree_ref[0].descendants

[Node("['EPEN-siletti', 'Ependymal-smajic', 'Ependymal-smajic']"),
 Node("['MISC-siletti']"),
 Node("['NEUR-siletti', 'NEURO-welch', 'NEURO-welch', 'Neuron-smajic', 'Neuron-smajic', 'Neuron-agarwal', 'Neuron-agarwal']"),
 Node("['OLIGO-siletti', 'Oligodendrocyte-altena', 'Oligodendrocyte-altena', 'OLIGO-welch', 'OLIGO-welch', 'Oligodendrocytes-smajic', 'Oligodendrocytes-smajic', 'ODC-agarwal']"),
 Node("['OPC-siletti', 'OPC-altena', 'OPC-altena', 'OPC-welch', 'OPC-welch', 'OPCs-smajic', 'OPCs-smajic', 'OPC-agarwal', 'OPC-agarwal']"),
 Node("['ENDO-welch']"),
 Node("['Astrocytes-smajic', 'Astrocytes-agarwal']"),
 Node("['Microglia-smajic']"),
 Node("['Endothelial cells-smajic']"),
 Node("['Pericytes-smajic']"),
 Node("['Endothelial-agarwal']")]

In [16]:
base_harmonized=harmonize_labels(adata=base,label_key='cell_type_lowest_res',tree=tree_ref,level=1,new_key='harmonized_lowest_res')

In [17]:
set(base_harmonized.obs['harmonized_lowest_res'].tolist())

{'Astrocytes-smajic',
 'ENDO-welch',
 'EPEN-siletti',
 'Endothelial cells-smajic',
 'Endothelial-agarwal',
 'MISC-siletti',
 'Microglia-smajic',
 'NEUR-siletti',
 'OLIGO-siletti',
 'OPC-siletti',
 'Pericytes-smajic',
 'unknown'}

(4) Predicting the labels can be done in multiple ways, this was what I saw in scarches tutorial and seems like a basic way. \
We subset the previously known cells for the training and then will reclassify all cells.

Filter out all unknown cells for the reference

In [18]:
from collections import Counter
Counter(base_harmonized.obs['harmonized_lowest_res'].tolist())

Counter({'unknown': 125426,
         'NEUR-siletti': 130101,
         'Microglia-smajic': 29104,
         'OLIGO-siletti': 175902,
         'Endothelial-agarwal': 72,
         'OPC-siletti': 25049,
         'Astrocytes-smajic': 29430,
         'ENDO-welch': 2905,
         'MISC-siletti': 2110,
         'EPEN-siletti': 479,
         'Pericytes-smajic': 652,
         'Endothelial cells-smajic': 990})

In [19]:
reference=base_harmonized[base_harmonized.obs['harmonized_lowest_res']!='unknown']

In [20]:
reference

View of AnnData object with n_obs × n_vars = 396794 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res', 'harmonized_lowest_res'
    

(4) Label prediction

In [21]:
## Copied from scArches git hub (https://github.com/theislab/scarches/blob/master/scarches/utils/knn.py#L10)
import scanpy as sc
import pandas as pd
import os
import numpy as np
import anndata
from sklearn.neighbors import KNeighborsTransformer

#These function were created by Lisa Sikemma

def weighted_knn_trainer(train_adata, train_adata_emb, n_neighbors=50):
    """Trains a weighted KNN classifier on ``train_adata``.
    Parameters
    ----------
    train_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to train KNN classifier with ``label_key`` as the target variable.
    train_adata_emb: str
        Name of the obsm layer to be used for calculation of neighbors. If set to "X", anndata.X will be
        used
    n_neighbors: int
        Number of nearest neighbors in KNN classifier.
    """
    print(
        f"Weighted KNN with n_neighbors = {n_neighbors} ... ",
        end="",
    )
    k_neighbors_transformer = KNeighborsTransformer(
        n_neighbors=n_neighbors,
        mode="distance",
        algorithm="brute",
        metric="euclidean",
        n_jobs=-1,
    )
    if train_adata_emb == "X":
        train_emb = train_adata.X
    elif train_adata_emb in train_adata.obsm.keys():
        train_emb = train_adata.obsm[train_adata_emb]
    else:
        raise ValueError(
            "train_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    k_neighbors_transformer.fit(train_emb)
    return k_neighbors_transformer    

def weighted_knn_transfer(
    query_adata,
    query_adata_emb,
    ref_adata_obs,
    label_keys,
    knn_model,
    threshold=1,
    pred_unknown=False,
    mode="package",
):
    """Annotates ``query_adata`` cells with an input trained weighted KNN classifier.
    Parameters
    ----------
    query_adata: :class:`~anndata.AnnData`
        Annotated dataset to be used to queryate KNN classifier. Embedding to be used
    query_adata_emb: str
        Name of the obsm layer to be used for label transfer. If set to "X",
        query_adata.X will be used
    ref_adata_obs: :class:`pd.DataFrame`
        obs of ref Anndata
    label_keys: str
        Names of the columns to be used as target variables (e.g. cell_type) in ``query_adata``.
    knn_model: :class:`~sklearn.neighbors._graph.KNeighborsTransformer`
        knn model trained on reference adata with weighted_knn_trainer function
    threshold: float
        Threshold of uncertainty used to annotating cells as "Unknown". cells with
        uncertainties higher than this value will be annotated as "Unknown".
        Set to 1 to keep all predictions. This enables one to later on play
        with thresholds.
    pred_unknown: bool
        ``False`` by default. Whether to annotate any cell as "unknown" or not.
        If `False`, ``threshold`` will not be used and each cell will be annotated
        with the label which is the most common in its ``n_neighbors`` nearest cells.
    mode: str
        Has to be one of "paper" or "package". If mode is set to "package",
        uncertainties will be 1 - P(pred_label), otherwise it will be 1 - P(true_label).
    """
    if not type(knn_model) == KNeighborsTransformer:
        raise ValueError(
            "knn_model should be of type sklearn.neighbors._graph.KNeighborsTransformer!"
        )

    if query_adata_emb == "X":
        query_emb = query_adata.X
    elif query_adata_emb in query_adata.obsm.keys():
        query_emb = query_adata.obsm[query_adata_emb]
    else:
        raise ValueError(
            "query_adata_emb should be set to either 'X' or the name of the obsm layer to be used!"
        )
    top_k_distances, top_k_indices = knn_model.kneighbors(X=query_emb)

    stds = np.std(top_k_distances, axis=1)
    stds = (2.0 / stds) ** 2
    stds = stds.reshape(-1, 1)

    top_k_distances_tilda = np.exp(-np.true_divide(top_k_distances, stds))

    weights = top_k_distances_tilda / np.sum(
        top_k_distances_tilda, axis=1, keepdims=True
    )
    cols = ref_adata_obs.columns[ref_adata_obs.columns.str.startswith(label_keys)]
    uncertainties = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    pred_labels = pd.DataFrame(columns=cols, index=query_adata.obs_names)
    for i in range(len(weights)):
        for j in cols:
            y_train_labels = ref_adata_obs[j].values
            unique_labels = np.unique(y_train_labels[top_k_indices[i]])
            best_label, best_prob = None, 0.0
            for candidate_label in unique_labels:
                candidate_prob = weights[
                    i, y_train_labels[top_k_indices[i]] == candidate_label
                ].sum()
                if best_prob < candidate_prob:
                    best_prob = candidate_prob
                    best_label = candidate_label

            if pred_unknown:
                if best_prob >= threshold:
                    pred_label = best_label
                else:
                    pred_label = "Unknown"
            else:
                pred_label = best_label

            if mode == "package":
                uncertainties.iloc[i][j] = (max(1 - best_prob, 0))

            else:
                raise Exception("Inquery Mode!")

            pred_labels.iloc[i][j] = (pred_label)

    print("finished!")

    return pred_labels, uncertainties

In [22]:
##K-nn model for label transfer/prediction (see https://docs.scarches.org/en/latest/hlca_map_classify.html)
#Used in Jupyter Notebook [Loosely-coupled/dependent]

knn_transformer_scVI = weighted_knn_trainer(
    train_adata=reference,
    train_adata_emb="X_scVI",  # Embedding
    n_neighbors=30,
)
labels_scVI, uncert_scVI = weighted_knn_transfer(
    query_adata=base_harmonized,
    query_adata_emb="X_scVI",  # Embedding
    label_keys="harmonized_lowest_res",  # labels to transfer
    knn_model=knn_transformer_scVI,
    ref_adata_obs=reference.obs,
)
uncertainty_threshold = 0.2
labels_scVI.rename(
    columns={
        "harmonized_lowest_res": "harmonized_lowest_res_unfiltered_scVI" # .obs feature with all predicted labels
    },
    inplace=True,
)
uncert_scVI.rename(
    columns={
        "harmonized_lowest_res": "harmonized_lowest_res_uncert_scVI" # .obs feature with uncertainty score
    },
    inplace=True,
)


Weighted KNN with n_neighbors = 30 ... finished!


In [23]:
base_harmonized.obs = base_harmonized.obs.join(labels_scVI)
base_harmonized.obs = base_harmonized.obs.join(uncert_scVI)

base_harmonized.obs['harmonized_lowest_res_filtered_02_scVI'] = base_harmonized.obs[ # .obs feature with uncertain cells filtered
    "harmonized_lowest_res_unfiltered_scVI"
].mask(
    base_harmonized.obs["harmonized_lowest_res_uncert_scVI"] > uncertainty_threshold,
    "Unknown",
)
print(
    f"Percentage of unknown for scVI, with uncertainty_threshold={uncertainty_threshold}:"
)
print(
    f"Filtered: {np.round(sum(base_harmonized.obs['harmonized_lowest_res_filtered_02_scVI'] =='Unknown')/base_harmonized.n_obs*100,2)}% of previously unknown"
)


Percentage of unknown for scVI, with uncertainty_threshold=0.2:
Filtered: 2.77% of previously unknown


In [24]:
base_harmonized

AnnData object with n_obs × n_vars = 522220 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res', 'harmonized_lowest_res', 'harmonized

The unscert .obs feature needs to be a string rather than float

In [25]:
base_harmonized.obs['harmonized_lowest_res_uncert_scVI']=base_harmonized.obs['harmonized_lowest_res_uncert_scVI'].astype(str)

In [53]:
base_harmonized.write('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/harmonized_base.h5ad')

Filter out the failed predictions.

In [28]:
set(base_harmonized.obs['harmonized_lowest_res_uncert_scVI'].tolist())

{'0.03357410430908203',
 '0.03331446647644043',
 '0.3333258032798767',
 '0.33325278759002686',
 '0.19986426830291748',
 '0.43336230516433716',
 '0.5323441028594971',
 '0.23596113920211792',
 '0.033261656761169434',
 '0.5000633299350739',
 '0.0666666030883789',
 '0.1667015552520752',
 '0.5665761828422546',
 '0.0998009443283081',
 '0.03307229280471802',
 '0.40015625953674316',
 '0.49959343671798706',
 '0.29941117763519287',
 '0.2000523805618286',
 '0.4998500347137451',
 '0.0999377965927124',
 '0.3999409079551697',
 '0.16660428047180176',
 '0.0666424036026001',
 '0.03483939170837402',
 '0.06668531894683838',
 '0.3667301535606384',
 '0.19902276992797852',
 '0.4655439257621765',
 '0.3657694458961487',
 '0.39990144968032837',
 '0.0333516001701355',
 '0.43128329515457153',
 '0.19997382164001465',
 '0.033457398414611816',
 '0.03337395191192627',
 '0.03316605091094971',
 '0.23331624269485474',
 '0.43326103687286377',
 '0.03251415491104126',
 '0.2999950647354126',
 '0.23352950811386108',
 '0.399

In [29]:
base_filtered=base_harmonized[base_harmonized.obs['harmonized_lowest_res_filtered_02_scVI']!="Unknown"]

In [30]:
base_filtered

View of AnnData object with n_obs × n_vars = 507763 × 45433
    obs: 'species', 'gender', 'age', 'instrument', 'technology', 'atlas', 'sample', 'batch_1', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'n_genes', 'total_counts_mito', 'total_counts_norm', 'original_Library', 'original_Brain_Region', 'original_Level_1_cell_type', 'original_Level_2_cell_type', 'original_cellclass_lvl1', 'original_subclass_lvl3_1', 'original_cellclass_lvl1_n', 'original_lineage', 'original_subclass_lvl2', 'original_celltype_lvl3', 'donor', 'original_supercluster_term', 'original_cluster_id', 'original_subcluster_id', 'original_celltype', 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lowest_res', 'cell_type_low_res', 'cell_type_med_res', 'cell_type_high_res', 'harmonized_lowest_res', 'ha

In [31]:
cell_type_low_res_batch=base_filtered.obs['harmonized_lowest_res_filtered_02_scVI'].tolist()
for i in set(cell_type_low_res_batch):
    c=0
    for j in cell_type_low_res_batch:
        if j == i:
            c+=1
    print(f'{i}: {c}')

ENDO-welch: 4394
MISC-siletti: 2377
OPC-siletti: 30575
EPEN-siletti: 563
OLIGO-siletti: 230390
Endothelial cells-smajic: 1400
Microglia-smajic: 41173
NEUR-siletti: 157244
Pericytes-smajic: 468
Astrocytes-smajic: 39179


In [32]:
base_filtered.write('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/harmonized_base_filtered.h5ad')

Use more generic labels

In [33]:
new_labels=[]
for i in base_filtered.obs['harmonized_lowest_res_filtered_02_scVI'].values.tolist():
    if i=='Astrocytes-smajic':
        new_labels.append('Astrocyte')
    elif i == 'ENDO-welch':
        new_labels.append('Endothelial')
    elif i == 'EPEN-siletti':
        new_labels.append('Ependymal')
    elif i == 'Endothelial cells-smajic':
        new_labels.append('Endothelial')
    elif i == 'MISC-siletti':
        new_labels.append('Miscellaneous')
    elif i == 'Microglia-smajic':
        new_labels.append('Microglia')
    elif i == 'NEUR-siletti':
        new_labels.append('Neuron')
    elif i == 'OLIGO-siletti':
        new_labels.append('Oligodendrocyte')
    elif i == 'OPC-siletti':
        new_labels.append('OPC')
    elif i == 'Pericytes-smajic':
        new_labels.append('Pericyte')
    elif i == 'Endothelial-agarwal':
        new_labels.append('Endothelial')
    else:
        new_labels.append('unknown')
base_filtered.obs['harmonized_lowest']=new_labels

<ipython-input-33-a54a50a97ff8>:27: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  base_filtered.obs['harmonized_lowest']=new_labels


In [35]:
base_filtered.obs['harmonized_lowest']

N3_TGACTAGTCATCACCC              Neuron
N3_GTTCGGGGTGTTCGAT     Oligodendrocyte
N3_TACCTTAAGTATGACA     Oligodendrocyte
N3_GTTACAGTCACATACG     Oligodendrocyte
N3_GGCGACTTCCTACAGA     Oligodendrocyte
                             ...       
TGGGAGATCACGGTCG-1-1        Endothelial
TGTTGGACATCTTCGC-1      Oligodendrocyte
TTACGTTGTCAACCTA-1            Microglia
TGTACAGAGGTGCGAT-1-1             Neuron
TTGCGTCGTCAAGCCC-1      Oligodendrocyte
Name: harmonized_lowest, Length: 507763, dtype: object

In [36]:
base_filtered.write('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/harmonized_base_filtered.h5ad')

Add Siletti metadata

In [47]:
siletti_metadata= pd.read_csv('/home/hers_basak/jjiang/jack/siletti_metadata_midbrain.csv')

In [48]:
names =base_filtered.obs['sample'].tolist()
index_names = []
for name in names:
    if name.replace('-','_') in siletti_metadata.iloc[:, 0].tolist():
        index_names.append(siletti_metadata.iloc[:, 0].tolist().index(name.replace('-','_')))
        
    else:
        index_names.append('')
abrev= [] 
dissect= [] 

for i in index_names:
    if i!='':
        abrev.append(siletti_metadata.iloc[i, 6])
        dissect.append(siletti_metadata.iloc[i, 5])
    else:
        abrev.append('')
        dissect.append('')
base_filtered.obs['siletti_dissection'] = dissect
base_filtered.obs['siletti_dissection_abrev'] = abrev

In [49]:
set(base_filtered.obs['siletti_dissection'].tolist())

{'',
 'Midbrain (M) - Inferior colliculus and nearby nuclei - IC',
 'Midbrain (M) - Periaqueductal gray and Dorsal raphe nucleus - PAG-DR',
 'Midbrain (M) - Periaqueductal gray and nearby nuclei - PAG',
 'Midbrain (M) - Pretectal region - PTR',
 'Midbrain (M) - Substantia Nigra - SN',
 'Midbrain (M) - Substantia Nigra, Red Nucleus, and nearby nuclei - SN-RN',
 'Midbrain (M) - Superior colliculus and nearby nuclei - SC',
 'Midbrain (RN) - Red Nucleus - RN'}

In [50]:
base_filtered.write('/home/hers_basak/jjiang/jack/for_next_person/data/scvi_model_final/harmonized_base_filtered.h5ad')

Depending on what you want to do, you can take the top N hvg.

In [65]:
sc.pp.highly_variable_genes(
    base,
    n_top_genes=5000,
    batch_key="atlas",
    subset=True)

/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/hers_basak/jjiang/jack/miniconda/envs/scarches2/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: T

In [68]:
base.write('/home/hers_basak/jjiang/jack/outputs/deliverables/5_integration/data/scvi/models/scvi_model_final/adata_predicted_lowest_filtered_5k.h5ad')